In [ ]:
#Arman Yashar Khojandi
#FAES BIOF509: Applied Machine Learning
#Profs Alexander Goncearenco & Ayal Gussow
#Due: 23 December 2018

## Overview
For this project, I analyzed a subset of 82 subjects (aged 22-35) from the WU-Minn Human Connectome Project 1200 Subjects Release 7T fMRI movie-watching protocol. 
-  Each subject had 4 total "sessions" where they watched video clips (no real-life footage, 2-4 minutes each, separated by 20 seconds of no stimulus). 
-  Each session had four such clips. 
-  All subjects completed self-report and standard-measurement psychological/emotional/cognitive tests, and behavioral measures were recorded and provided by HCP.
-  I preprocessed the data and compared the performance of sk-learn's PLS and Ridge regression procedures in predicting subjects' behavioral measures based on their neural responses to the stimuli--broken into 1.7 million cubic measures of brain space, hereafter termed "voxels."

## Some context
-  For this project I only used 82 subjects (total 184), 2 video clips, and 3 behaviors in analysis
-  The video clips differed greatly in content
> -  "Dreary" consisted of rainy, desolate scenes without dialogue, humans, or a narrative, and containing eerie music
> -  "Bridgeville" was a clip about oldtown USA wherein many people were interviewed and described their lifestyles and traditions, somewhat nostalgic and happy in tone, sentimental music in the background
-  The three behaviors I used were scores of 
> -  "Sadness_Unadj" (self-reported), a measure of how sad subjects were
> -  "EmotSupp_Unadj" (self-reported, a measure of how confident subjects were that people in their lives would support them emotionally/cared about them)
> -  "PMAT24_A_CR" (standardized) a measure of fluid intelligence; how many correct responses they had on the PMAT24



## Preprocessing & ML stuff I did here
-  HCP provided the data with ICA-FIX denoising and preprocessing already performed 
-  I performed spatial blurring of the data (1.6mm^3 voxels --> 4.0mm^3 voxels)
-  Initial subject timeseries were read in as (113, 136, 113, [time]) matrices--once steps were performed time axis collapsed
-  For each clip, I created a total sum of all subject timseries
> -  I then excluded each subject from the total and calculated the correlation between that subject and the mean of the subject-excluded total sum
- I concatenated the resulting data for all subjects (per clip)
- These data were read in below under variable name "rnd1_fname" and input to the ML models as "X" after the important data were extracted
- The vector of behavioral measures for all subjects was read in as "y"
> -  1 subject did not have data for "Dreary" clip, so 81 subjects were used for that one
- I performed a standard scalar and transform procedure on data beforehand for better visualization/interpretation of results
- Transformed X was then fit to transformed y using cross-validation (cross_val_score and cross_val_predict used to determine relationship and predict y from X.
- Finally, I plotted y_observed vs y_predicted
- CV R-squared values from each fold were saved to output files, as were the output vectors of y_obs, y_preds
- I duplicated the for loop for Ridge regression and PLS because at the time I was experiencing performance issues and wanted to parallelize so to speak

-  NOTE: In the second pass after initial results, following z-scoring, I weighted each subject's data by his/her behavioral measure of interest to see the models were better able to fit features to subjs--that is why there are two versions for each plot of clipxbehavior in my powerpoint
- the ML steps were the same as below, however
- I also copy pasted this from a couple of very ugly, cluttered notebooks, so I apologize if some small snippet of code is missing--you can rest assured I did it since I got results...

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import scipy as sp
import seaborn as sns
import statsmodels.api as sm
import nibabel as nib
import nilearn

from pathlib import Path
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import cross_val_score
from sklearn.cross_decomposition import PLSRegression

from scipy.stats import linregress, zscore
from matplotlib.axes import Axes
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline

In [ ]:
#Reading in behavioral measures to be used as y
behav_data = pd.read_csv('/data/HCP_preproc/just_numeric_measures.csv', index_col=['Subject'])
behav_data.head()


In [ ]:
#Preparing small subset of behaviors and clips that I will analyze
behaviors = [ "Sadness_Unadj", "EmotSupp_Unadj", "PMAT24_A_CR" ]
listclips = [ "bridgeville", "dreary"]

print(listclips)
print(behaviors)

In [ ]:
#Machine learnin' time! ...Will the cognition engine boot up?

g = open(clipdir + "PLS_outputs.txt", 'a')
for clip in listclips:
    
    #Grabbing file of interest
    rnd1_fname = "{}{}.nii".format(clipdir, clip) # read in nifti file--in the second pass this was something like ""{}{}_{}_corrcoef_wmean_stacked.npy" --a 2D array of brain data for all subjects
    rnd1_nb = nib.load(rnd1_fname) #This was the step that caused performance issues--nibabel and nilearn will memory map for larger files unless told mmap=False, causing read/write issues known to exist on the latest versions of helix/felix
    rnd1_img_data = nib.load(rnd1_fname).get_fdata() #Extract data
    x, y, z, subjlen = rnd1_img_data.shape #get dimensions for later
    #Redone for sanity check later
    img_data = rnd1_nb.get_fdata()
    i_size, j_size, k_size, t = img_data.shape
    
    X = img_data.reshape(t,-1) # this gets the feature matrix into the correct shape (subjects x features)
    
    #Loop for each behavioral measure--Austin Powers reference!
    for oh_behave in behaviors:
        y = behav_data[oh_behave]
        y = y[0:82]
        
        #In the case of "dreary" which only has data for 81 subjects
        if subjlen == 81:
            y = y.drop(index=65) #not super elegant/flexible--I changed this later to drop data for any subject who didn't have the neuroimaging data
        else:
            y = y[0:82]
        y = np.reshape(np.array(y),(-1,1)) # this gets the target matrix into the correct shape (subjects x target(s))
        print("The shape of X is " + str(X.shape))
        print("The shape of y is " + str(y.shape))
        
        #Define scaler/normalized data
        scalar = preprocessing.StandardScaler()

        X_scaler = preprocessing.StandardScaler().fit(X)
        y_scaler = preprocessing.StandardScaler().fit(y)

        X_scaled = X_scaler.transform(X) 
        y_scaled = y_scaler.transform(y)
        
        #Output to file
        g.write("Clip is {} \n".format(clip))
        g.write("Behavior is {} \n".format(oh_behave))
        g.flush()
        
        #Partial least squares regression
        pls2 = PLSRegression(n_components=1)
        pls2.fit(X_scaled,y_scaled)
        pls2_R2 = pls2.score(X_scaled,y_scaled)
        print("pls2 R^2 = {}".format(pls2_R2))
        g.write("PLS R^2 = {} \n".format(pls2_R2))
        g.flush()
        
        pls2_pipeline = Pipeline([('transformer', scalar), ('estimator', pls2)]) #set up the pipeline for less clutter
        
        #Run the model with tenfold cross validation
        pls_cross_scores = cross_val_score(pls2_pipeline, X, y, cv=10, scoring="r2")
        g.write("PLS cross val scores are {} \n".format(pls_cross_scores))
        g.flush()
        
        #Predict y
        pls_pred = cross_val_predict(pls2_pipeline, X, y, cv=10)
        g.write("PLS predicted values are {} \n".format(pls_pred))
        pls_pred_z = zscore(pls_pred)
        g.write("Normalized PLS predicted values are: {} \n".format(pls_pred_z))
        g.flush()
        
        #Plot the results
        fig, ax = plt.subplots()
        ax.scatter(y_scaled, pls_pred_z, edgecolors=(0, 0, 0))        
        plt.axis('scaled')
        axes = plt.gca()
        
        #Try to doctor the axis scales a bit to look nicer
        if (y_scaled.max() - y_scaled.min()) > (pls_pred_z.max() - pls_pred_z.min()):
            ax.set_ylim([y_scaled.min() - 1, y_scaled.max() + 1])
            ax.set_xlim([y_scaled.min() - 1, y_scaled.max() + 1])
        else:
            ax.set_ylim([pls_pred_z.min() - 1, pls_pred_z.max() + 1])
            ax.set_xlim([pls_pred_z.min() - 1, pls_pred_z.max() + 1])
        #axes.set_aspect('equal', adjustable='box', anchor='C')
        #axes.set_anchor('C')
        
        #Get stats to add R^2 value in legend--this wasn't done in the first pass but I added it here since I'm not marking down the later versions
        slope, intercept, rval, pval, stderr = linregress(y_scaled[:,0], pls_pred_z[:,0])
        ax.plot(y_scaled, slope*y_scaled[:,0] + intercept, 'k--', label='R^2: {0:.2f}'.format(rval), lw=2)
        plt.legend()
        ax.set_title("PLS {}".format(clip))
        ax.set_xlabel("{} measured".format(oh_behave))
        ax.set_ylabel("{} predicted".format(oh_behave))
        plt.show()
        fig.savefig("{}/PLS_{}_{}".format(clipdir, clip, oh_behave))
g.close()

In [ ]:
#Same method but with Ridge regression (assigned lambda as well as chosen via cross-val procedure)

In [ ]:
h = open("{}/new_alphas_10_100_1000_clf_and_clfCV_results.txt".format(clipdir), "a")
for clip in listclips:
    rnd1_fname = "{}{}.nii".format(clipdir, clip)
    rnd1_nb = nib.load(rnd1_fname)
    rnd1_img_data = nib.load(rnd1_fname).get_fdata()
    x, y, z, subjlen = rnd1_img_data.shape
    img_data = rnd1_nb.get_fdata()
    i_size, j_size, k_size, t = img_data.shape
    X = img_data.reshape(t,-1) # this gets the feature matrix into the correct shape (subjects x features)
    for oh_behave in behaviors:
        y = behav_data[oh_behave]
        y = y[0:82]
        if subjlen == 81:
            y = y.drop(index=65)
        else:
            y = y[0:82]
        y = np.reshape(np.array(y),(-1,1)) # this gets the target matrix into the correct shape (subjects x target(s))
        print("The shape of X is " + str(X.shape))
        print("The shape of y is " + str(y.shape))
        
        #Define scaler/normalized data
        scalar = preprocessing.StandardScaler()

        X_scaler = preprocessing.StandardScaler().fit(X)
        y_scaler = preprocessing.StandardScaler().fit(y)

        X_scaled = X_scaler.transform(X) 
        y_scaled = y_scaler.transform(y)
        
        #Output to file
        h.write("Clip is {} \n".format(clip))
        h.write("Behavior is {} \n".format(oh_behave))
        h.flush()
            
        #CLF Ridge regression    
        clf = Ridge(alpha=100)
        clf.fit(X_scaled, y_scaled)
        clf_R2 = clf.score(X_scaled,y_scaled)
        print("clf R^2 = {}".format(clf_R2))
        h.write("CLF R^2 = {} \n".format(clf_R2))
        h.flush()
        
        clf_pipeline = Pipeline([('transformer', scalar), ('estimator', clf)])
        
        clf_cross_scores = cross_val_score(clf_pipeline, X, y, cv=10, scoring="r2")
        h.write("CLF cross val scores are: {} \n".format(clf_cross_scores))
        h.flush()
        
        clf_pred = cross_val_predict(clf_pipeline, X, y, cv=10)
        h.write("CLF predicted values are: {} \n".format(clf_pred))
        clf_pred_z = zscore(clf_pred)
        h.write("Normalized CLF predicted values are: {} \n".format(clf_pred_z))
        h.flush()
        
        fig, ax = plt.subplots()
        ax.scatter(y_scaled, clf_pred_z, edgecolors=(0, 0, 0))
        plt.axis('scaled')
        axes = plt.gca()
        if (y_scaled.max() - y_scaled.min()) > (clf_pred_z.max() - clf_pred_z.min()):
            ax.set_ylim([y_scaled.min() - 1, y_scaled.max() + 1])
            ax.set_xlim([y_scaled.min() - 1, y_scaled.max() + 1])
        else:
            ax.set_ylim([clf_pred_z.min() - 1, clf_pred_z.max() + 1])
            ax.set_xlim([clf_pred_z.min() - 1, clf_pred_z.max() + 1])
        #axes.set_aspect('equal', adjustable='box', anchor='C')
        #axes.set_anchor('C')
        slope, intercept, rval, pval, stderr = linregress(y_scaled[:,0], clf_pred_z[:,0])
        ax.plot(y_scaled, slope*y_scaled[:,0] + intercept, 'k--', label='R^2: {0:.2f}'.format(rval), lw=2)
        plt.legend()
        ax.set_title("CLF_Ridge {}".format(clip))
        ax.set_xlabel("{} measured".format(oh_behave))
        ax.set_ylabel("{} predicted".format(oh_behave))
        plt.show()
        fig.savefig("{}/clf_{}_{}".format(clipdir, clip, oh_behave))
        
         #CLF Ridge regression cross-validated
        clfCV = RidgeCV(alphas=np.array([ 10., 100., 1000. ]), cv=10)
        clfCV.fit(X_scaled, y_scaled)
        clfCV_R2 = clfCV.score(X_scaled,y_scaled)
        print("clfCV R^2 = {}".format(clfCV_R2))
        h.write("CLFCV R^2 is {} \n".format(clfCV_R2))
        h.flush()
        
        clfCV_pipeline = Pipeline([('transformer', scalar), ('estimator', clfCV)])
        
        cross_scores = cross_val_score(clfCV_pipeline, X, y, cv=10, scoring="r2")
        h.write("CLFCV cross val scores: {} \n".format(cross_scores))
        h.flush()
        
        clfCV_pred = cross_val_predict(clfCV_pipeline, X, y, cv=10)
        h.write("clfCV predicted values: {} \n".format(clfCV_pred))
        clfCV_pred_z = zscore(clfCV_pred)
        h.write("Normalized clfCV predicted values are: {} \n".format(clfCV_pred_z))
        h.flush()
        
        fig, ax = plt.subplots()
        ax.scatter(y_scaled, clfCV_pred_z, edgecolors=(0, 0, 0))
        plt.axis('scaled')
        axes = plt.gca()
        if (y_scaled.max() - y_scaled.min()) > (clfCV_pred_z.max() - clfCV_pred_z.min()):
            ax.set_ylim([y_scaled.min() - 1, y_scaled.max() + 1])
            ax.set_xlim([y_scaled.min() - 1, y_scaled.max() + 1])
        else:
            ax.set_ylim([clfCV_pred_z.min() - 1, clfCV_pred_z.max() + 1])
            ax.set_xlim([clfCV_pred_z.min() - 1, clfCV_pred_z.max() + 1])
        #axes.set_aspect('equal', adjustable='box', anchor='C')
        #axes.set_anchor('C')
        slope, intercept, rval, pval, stderr = linregress(y_scaled[:,0], clfCV_pred_z[:,0])
        ax.plot(y_scaled, slope*y_scaled[:,0] + intercept, 'k--', label='R^2: {0:.2f}'.format(rval), lw=2)
        plt.legend()
        ax.set_title("CLF_RidgeCV_{}".format(clip))
        ax.set_xlabel("{} measured".format(oh_behave))
        ax.set_ylabel("{} predicted".format(oh_behave))
        plt.show()
        fig.savefig("{}/clfCV_{}_{}".format(clipdir, clip, oh_behave))
h.close()

## Second pass with behavior-weighted mean
Because it's Christmas, I've included the second run through of the data that produced better results, with some apparent differences in the code, though the "high-flow logic" is mostly unchanged

In [ ]:
groupdir = '/data/HCP_preproc/7T_movie/SubjectData/GroupResults/'
behaviors = [ 'EmotSupp_Unadj', 'Sadness_Unadj', 'PMAT24_A_CR' ]
listclips = [ 'dreary', 'bridgeville' ]

h = open("{}/weighted_mean_alphas_10_100_1000_ridge_and_ridgeCV_results.txt".format(groupdir), "a") #output file for values
for clip in listclips:
    #rnd1_fname = "{}weighted_mean_{}_{}_stacked_corr.nii".format(corrdir, clip, oh_behave) #stacked correlation matrix for all subjects
    #rnd1_nb = nib.load(rnd1_fname)
    #rnd1_img_data = nib.load(rnd1_fname).get_fdata()
    #x, y, z, subjlen = rnd1_img_data.shape
    #img_data = rnd1_nb.get_fdata()
    #i_size, j_size, k_size, t = img_data.shape
    #X = img_data.reshape(t,-1) # this gets the feature matrix into the correct shape (subjects x features)
    for oh_behave in behaviors:
        X = np.load("{}{}_{}_corrcoef_wmean_stacked.npy".format(groupdir, clip, oh_behave))
        brain, subjlen = X.shape
        

        print("Shape of corrcoef matrix is ({},{})".format(brain, subjlen))
        y = behav_data[oh_behave]
        y = y[0:82]
        if subjlen == 81:
            y = y.drop(index=65)
        else:
            y = y[0:82]
        y = np.reshape(np.array(y),(-1,1)) # this gets the target matrix into the correct shape (subjects x target(s))
        print("The shape of X is " + str(X.shape))
        print("The shape of y is " + str(y.shape))
     
        #X = X.astype(np.float)
        #X = X[np.isfinite(X)]
        #y = y.astype(np.float)

        print(type(X))
        print(type(y))
        
        #Define scaler/normalized data
        scalar = preprocessing.StandardScaler()

        X_scaler = preprocessing.StandardScaler().fit(X)
        y_scaler = preprocessing.StandardScaler().fit(y)

        X_scaled = X_scaler.transform(X) 
        y_scaled = y_scaler.transform(y)
        
        #Output to file
        h.write("Clip is {} \n".format(clip))
        h.write("Behavior is {} \n".format(oh_behave))
        h.flush()
            
        #CLF Ridge regression    
        clf = Ridge(alpha=100)
        clf.fit(X_scaled, y_scaled)
        clf_R2 = clf.score(X_scaled,y_scaled)
        print("clf R^2 = {}".format(clf_R2))
        h.write("CLF R^2 = {} \n".format(clf_R2))
        h.flush()
        
        #Run ridge pipeline and cross validate and predict
        clf_pipeline = Pipeline([('transformer', scalar), ('estimator', clf)])
        
        clf_cross_scores = cross_val_score(clf_pipeline, X, y, cv=10, scoring="r2")
        h.write("CLF cross val scores are: {} \n".format(clf_cross_scores))
        h.flush()
        
        clf_pred = cross_val_predict(clf_pipeline, X, y, cv=10)
        h.write("CLF predicted values are: {} \n".format(clf_pred))
        clf_pred_z = zscore(clf_pred)
        h.write("Normalized CLF predicted values are: {} \n".format(clf_pred_z))
        h.flush()
        
        #Plot observed vs predicted values
        fig, ax = plt.subplots()
        ax.scatter(y_scaled, clf_pred_z, edgecolors=(0, 0, 0))
        plt.axis('scaled')
        axes = plt.gca()
        if (y_scaled.max() - y_scaled.min()) > (clf_pred_z.max() - clf_pred_z.min()):
            ax.set_ylim([y_scaled.min() - 1, y_scaled.max() + 1])
            ax.set_xlim([y_scaled.min() - 1, y_scaled.max() + 1])
        else:
            ax.set_ylim([clf_pred_z.min() - 1, clf_pred_z.max() + 1])
            ax.set_xlim([clf_pred_z.min() - 1, clf_pred_z.max() + 1])
        #axes.set_aspect('equal', adjustable='box', anchor='C')
        #axes.set_anchor('C')
        slope, intercept, rval, pval, stderr = linregress(y_scaled[:,0], clf_pred_z[:,0])
        ax.plot(y_scaled, slope*y_scaled[:,0] + intercept, 'k--', label='R^2: {0:.2f}'.format(rval), lw=2)
        plt.legend()
        ax.set_title("CLF_Ridge {}".format(clip))
        ax.set_xlabel("{} measured".format(oh_behave))
        ax.set_ylabel("{} predicted".format(oh_behave))
        plt.show()
        fig.savefig("{}/weighted_mean_ridge_{}_{}.png".format(groupdir, clip, oh_behave)) #save plot
        
        #CLF Ridge regression cross-validated
        clfCV = RidgeCV(alphas=np.array([ 10., 100., 1000. ]), cv=10)
        clfCV.fit(X_scaled, y_scaled)
        clfCV_R2 = clfCV.score(X_scaled,y_scaled)
        print("clfCV R^2 = {}".format(clfCV_R2))
        h.write("CLFCV R^2 is {} \n".format(clfCV_R2))
        h.flush()
        
        clfCV_pipeline = Pipeline([('transformer', scalar), ('estimator', clfCV)])
        
        cross_scores = cross_val_score(clfCV_pipeline, X, y, cv=10, scoring="r2")
        h.write("CLFCV cross val scores: {} \n".format(cross_scores))
        h.flush()
        
        clfCV_pred = cross_val_predict(clfCV_pipeline, X, y, cv=10)
        h.write("clfCV predicted values: {} \n".format(clfCV_pred))
        clfCV_pred_z = zscore(clfCV_pred)
        h.write("Normalized clfCV predicted values are: {} \n".format(clfCV_pred_z))
        h.flush()
        
        #Plot observed vs. predicted values
        fig, ax = plt.subplots()
        ax.scatter(y_scaled, clfCV_pred_z, edgecolors=(0, 0, 0))
        plt.axis('scaled')
        axes = plt.gca()
        if (y_scaled.max() - y_scaled.min()) > (clfCV_pred_z.max() - clfCV_pred_z.min()):
            ax.set_ylim([y_scaled.min() - 1, y_scaled.max() + 1])
            ax.set_xlim([y_scaled.min() - 1, y_scaled.max() + 1])
        else:
            ax.set_ylim([clfCV_pred_z.min() - 1, clfCV_pred_z.max() + 1])
            ax.set_xlim([clfCV_pred_z.min() - 1, clfCV_pred_z.max() + 1])
        #axes.set_aspect('equal', adjustable='box', anchor='C')
        #axes.set_anchor('C')
        slope, intercept, rval, pval, stderr = linregress(y_scaled[:,0], clfCV_pred_z[:,0])
        ax.plot(y_scaled, slope*y_scaled[:,0] + intercept, 'k--', label='R^2: {0:.2f}'.format(rval), lw=2)
        plt.legend()
        ax.set_title("CLF_RidgeCV_{}".format(clip))
        ax.set_xlabel("{} measured".format(oh_behave))
        ax.set_ylabel("{} predicted".format(oh_behave))
        plt.show()
        fig.savefig("{}/weighted_mean_clfCV_{}_{}.png".format(groupdir, clip, oh_behave)) #save plot
h.close()